<a href="https://colab.research.google.com/github/Modequick/Machine-Learning-Project/blob/main/Skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import datetime
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [181]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [182]:
df=pd.read_csv("/content/drive/MyDrive/Skripsi/Change_Accom_Behavior_Hashed 2.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Improving Data Logic


In [184]:
df.dtypes

profile_id                         object
order_id                           object
accom_id                           object
room_id                            object
accom_chain_name                   object
selling_price_idr                 float64
gbv_idr                           float64
coupon_idr                          int64
sales_date_utc7                    object
sales_timestamp                    object
month                               int64
check_in_date                      object
check_out_date                     object
accom_region                       object
accom_star_rating                   int64
interface                          object
num_rooms                           int64
is_use_coupon                      object
last_issued_date                   object
accom_type                         object
room_type                          object
bed_type                           object
room_size_inmeter                 float64
pid_score                         

#Exploratory Data Analysis

### Descriptive Statistic

In [ ]:
df.describe()

### Data Cleaning

In [ ]:
#missing_value
missing_value = df.isnull().sum() *100 / len(df) 

df_missing_value = pd.DataFrame( {"missing_count" : df.isnull().sum(),
             "percent_missing" : missing_value} )
print(df_missing_value.sort_values(by = "missing_count",ascending=False).head(20))

print("length of columns are : ", len(df.columns))

In [ ]:

#selecing columns that are not in the list 
df_clean = df.loc[:, ~df.columns.isin(df_missing_value.sort_values(by = "missing_count",ascending=False).head(20).index)]


# droping non intuitive columns 
df_clean = df_clean.loc[:, ~df_clean.columns.isin(["count_buy_that_day",
                                                   "count_buy_that_month",
                                                   "count_buy_that_next_day",
                                                   "room_id",
                                                   "order_id",
                                                   "profile_id",
                                                   "interface",
                                                   "last_issued_date",
                                                   "next_bed_type",
                                                   "month",
                                                   "next_time_stamp",
                                                   "sales_timestamp",
                                                   "next_accom_id",
                                                   "next_is_use_coupon",
                                                   "next_num_rooms",
                                                   "next_gbv_idr",
                                                   "accom_id" ,
                                                   "check_in_date",
                                                   "check_out_date",
                                                   "sales_date_utc7",
                                                   "next_check_in_date",
                                                   "next_check_out_date",
                                                   "next_sales_date",
                                                   "next_is_use_coupon",
                                                   "is_use_coupon",
                                                   "next_coupon_idr",
                                                   "coupon_idr"
                                                   ])]
                                                   
#next_check_in_date,next_check_out_date,"next_check_in_date"

In [ ]:
missing_value = df_clean.isnull().sum() *100 / len(df) 
df_missing_value = pd.DataFrame( {"missing_count" : df_clean.isnull().sum(),
             "percent_missing" : missing_value} )
print("length columns : ",len(df_clean.columns)  )
print(df_missing_value.sort_values(by = "missing_count",ascending=False))


### Data Transformation 

In [ ]:
df=df_clean 
# df_trial  = df.head(1000)
df.head()

In [ ]:
#Data Transformation : Label encoding 

accom_region_value = df.groupby(by = ["accom_region"],as_index = False).agg({"selling_price_idr" : 'mean'}).rename(columns={'selling_price_idr':'accom_region_value'})
accom_type_value = df.groupby(by = ["accom_type"],as_index = False).agg({"selling_price_idr" : 'mean'}).rename(columns={'selling_price_idr':'accom_type_value'})
next_accom_region_value = df.groupby(by = ["next_accom_region"],as_index = False).agg({"next_selling_price" : 'mean'}).rename(columns={'next_selling_price':'next_accom_region_value'})
next_accom_type_value = df.groupby(by = ["next_accom_type"],as_index = False).agg({"next_selling_price" : 'mean'}).rename(columns={'next_selling_price':'next_accom_type_value'})

df = df.merge(accom_region_value, on = "accom_region", how = "left")
df = df.merge(accom_type_value, on = "accom_type", how = "left")
df = df.merge(next_accom_region_value, on = "next_accom_region", how = "left")
df = df.merge(next_accom_type_value, on = "next_accom_type", how = "left")
df = df.loc[:, ~df.columns.isin(["accom_type","accom_region","next_accom_type","next_accom_region"])]
df ["next_hotel_score"] = df["next_hotel_score"].fillna(df["next_hotel_score"].mean())
df ["hotel_score"] = df["hotel_score"].fillna(df["hotel_score"].mean())
df ["accom_type_value"] = df["accom_type_value"].fillna(df["accom_type_value"].median())
df.head()



In [ ]:
#Data Transformation : Scaling 
def build_min_max_scaler(df) :
  scaled_columns = pd.DataFrame()
  for i in df.columns:
      min = df[i].min()
      df[i] = df[i]-min
      max = df[i].max()
      scaled_columns[i] = pd.DataFrame( round(df[i]/max ,5) )
  return scaled_columns

df = build_min_max_scaler(df)    
df.head()

#Feature Engineering 

##K-MEANS 

In [ ]:
y = df[["next_accom_star_rating","next_selling_price","next_hotel_score","next_accom_region_value","next_accom_type_value"]]
X = df.loc[:, ~df.columns.isin(y.index)]

In [ ]:
y.head()

In [ ]:
results = []

for i in range (1, 6):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(y)
    results.append(kmeans.inertia_)   

def index_for_kmeans(results) : 
  b=[]
  a = 0 
  for i in results:
    a = a+ 1  
    b.append(a)
  return b     

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(index_for_kmeans(results), results, marker='o', linestyle='--')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of squared distance')
plt.show()



In [ ]:
#Using Elbow method, we found the best model for this is 2 
kmeans = KMeans(n_clusters = 3 , init = 'k-means++',random_state = 42 )
kmeans.fit(y)
y = kmeans.labels_

In [ ]:
pd.Index( y.tolist()).value_counts()

#Classification Modeling

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#splitting the dataset 
X_train, X_test, y_train,y_test= train_test_split(X,y, test_size = 0.2 , random_state = 42)

In [ ]:
#Random Forest 

rfc = RandomForestClassifier(random_state = 0 )

#fit to model 
rfc.fit(X_train,y_train)

y_pred = rfc.predict(X_test)





In [ ]:
accuracy_score(y_test, y_pred)